In [13]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [21]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'd-pis-model'
id_name = 'Id'
target = 'Choice'

file_name = "./d-pis/test_Influencers in Social Networks.csv"
test_file_name = "./d-pis/test_Influencers in Social Networks-no-label.csv"

kaggle_file_name = "./d-pis/kaggle-test-Influencers in Social Networks.csv"
kaggle_test_file_name = "./d-pis/kaggle-test-Influencers in Social Networks-no-label.csv"

kaggle_pred_file_name = "./d-pis/kaggle-test-Influencers in Social Networks-predictions.csv"

In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

   A_follower_count  A_following_count  A_listed_count  A_mentions_received  \
0           6085071                 67           30758         10357.245163   
1              3741               1635             242             0.565491   
2               867                413              52            10.906349   
3               279                 40              15             0.294044   
4              5996                906             192            15.050208   

   A_retweets_received  A_mentions_sent  A_retweets_sent   A_posts  \
0          5215.953047         0.100503         0.100503  0.100503   
1             0.316668         0.100503         0.100503  0.351840   
2             2.359024         3.975177         1.124418  2.635947   
3             0.294044         0.100503         0.100503  0.100503   
4             3.330828         0.837169         0.349327  0.588346   

   A_network_feature_1  A_network_feature_2  ...  B_following_count  \
0                32871           

In [16]:
f = open(test_file_name)

count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    pred_pos_prpb.append(pos_prob)
    count = count + 1


In [19]:
print(pred_label[:10])
print(pred_neg_prpb[:10])
print(pred_pos_prpb[:10])

import pandas as pd
import numpy as np

pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print("accuracy = {}".format(np.mean(y_true==y_pred)))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_pos_prpb, pos_label=1)
print("auc = {}".format(metrics.auc(fpr, tpr)))

[0, 1, 1, 0, 1, 0, 1, 1, 0, 0]
[0.9984276110772043, 0.024759650230407715, 0.15189796686172485, 0.9975453952793032, 0.09736382961273193, 0.995651709381491, 1.2040138244628906e-05, 0.005163669586181641, 0.8260233104228973, 0.9147557765245438]
[0.0015723889227956533, 0.9752403497695923, 0.8481020331382751, 0.002454604720696807, 0.9026361703872681, 0.004348290618509054, 0.9999879598617554, 0.9948363304138184, 0.17397668957710266, 0.08524422347545624]
accuracy = 0.8654545454545455
auc = 0.935562579348286


In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

#kaggle_test_ids = np.array(df[id_name])
#df = df.drop([id_name],axis=1) 

print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")

    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    pred_pos_prpb.append(pos_prob)   
    
    test_id = count
    #pred = int(response[0])
    kaggle_pred.append([test_id, pos_prob])   
    count = count + 1

print(kaggle_pred[0:10])
print(pred_neg_prpb[:10])
print(pred_pos_prpb[:10])


kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

   A_follower_count  A_following_count  A_listed_count  A_mentions_received  \
0            614689                400            5729           199.618296   
1            739581              10723            1279           102.026750   
2              3638               3341              85             2.793577   
3               613               1259              31             1.326041   
4           1352348                331           15647         19129.848662   

   A_retweets_received  A_mentions_sent  A_retweets_sent    A_posts  \
0            67.812469         9.366192         0.359534   4.094488   
1            96.771319         1.537322         1.054358   3.467754   
2             1.402703         2.594089         0.598150   3.802143   
3             0.356943         6.249678         2.323064   7.488116   
4         14552.733991         2.065418         0.580955  16.281502   

   A_network_feature_1  A_network_feature_2  ...  B_following_count  \
0                  669     

In [30]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3b962f0e-0785-4f4a-bdbc-7d8e3a7f9b60',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3b962f0e-0785-4f4a-bdbc-7d8e3a7f9b60',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 23 Mar 2021 16:51:39 GMT'},
  'RetryAttempts': 0}}